In [1]:
from circuit import gen_random_circuits
from circuit.formatter import qiskit_to_my_format_circuit, layered_circuits_to_qiskit, get_layered_instructions
from upstream import RandomwalkModel
from collections import defaultdict

import math
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend, gen_linear_topology, get_linear_neighbor_info, gen_fulllyconnected_topology

from qiskit import QuantumCircuit
import pennylane as qml
import numpy as np
from downstream.synthesis.pennylane_op_jax import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op_jax import layer_circuit_to_matrix

from scipy.stats import unitary_group

from downstream.synthesis.synthesis_model_pca_unitary_jax import find_parmas, pkl_dump, pkl_load, matrix_distance_squared, SynthesisModel, synthesize
from itertools import combinations
import time
from qiskit import transpile
import random
import cloudpickle as pickle
from circuit.formatter import layered_circuits_to_qiskit, to_unitary
from qiskit.quantum_info import Operator
from jax import vmap
from utils.unitaries import qft_U, grover_U


In [ ]:

n_qubits = 3
topology = gen_fulllyconnected_topology(n_qubits)
neigh_info = gen_fulllyconnected_topology(n_qubits)

# topology = gen_linear_topology(n_qubits)
# neigh_info = get_linear_neighbor_info(n_qubits, 1)
synthesis_data_path = f'./temp_data/{n_qubits}_synthesis_data.pkl'
regen = False
if regen:
    backend = Backend(n_qubits=n_qubits, topology=topology, neighbor_info=neigh_info, basis_single_gates=['u'],
                    basis_two_gates=['cz'], divide=False, decoupling=False)


    min_gate, max_gate = 2 * 4**n_qubits - 200, 2 * 4**n_qubits
    dataset = gen_random_circuits(min_gate=100, max_gate=max_gate, gate_num_step=max_gate//50, n_circuits=50,
                                    two_qubit_gate_probs=[2, 5], backend=backend, reverse=False, optimize=True, multi_process=True)


    upstream_model = RandomwalkModel(1, 20, backend)
    upstream_model.train(dataset, multi_process=True, remove_redundancy=False)
    synthesis_model = SynthesisModel(upstream_model, f'synthesis_{n_qubits}')
    data = synthesis_model.construct_data(dataset, multi_process = True)
    synthesis_model.construct_model(data)
    synthesis_model.save()
    
    with open(synthesis_data_path, 'wb') as f:
        pickle.dump(dataset, f)
else:
    synthesis_model: SynthesisModel = SynthesisModel.load(f'synthesis_{n_qubits}')
    backend: Backend = synthesis_model.backend

    with open(synthesis_data_path, 'rb') as f:
        Us, Vs = pickle.load(f)

In [ ]:
U = unitary_group.rvs(2**n_qubits)
dists = vmap(matrix_distance_squared, in_axis = (None, 0))(U, Us)